In [65]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [66]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape

(1000, 8)

In [67]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [68]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1]

In [69]:
X.isnull().sum() # Somar os atributos que o a nulos para cada coluna

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [70]:
mediana = X['SALDO_ATUAL'].median() # Podemos preencher os valores nulos com a mediana que não esta sucetivel a valores extremos.
# Ela não se influencia por valores muito acima ou abaixo da média, assim calculamos a mediana e preenchemos os valores nulo com ela.
# Isso é feito porque o modelo não considera linha por linha e sim pelo coletivo.
# Isso NÂO deve ser feito no banco de dados de produção. È feito apenas para treinamento do modelo.
mediana

np.float64(2323.0)

In [71]:
X['SALDO_ATUAL'].fillna(mediana,inplace=True)
# inplace=True -> Ele não vai atualizar os arquivos originais sem isso, ele só vai gerar um conjunto de dados com o valor
X.isnull().sum()

/tmp/ipykernel_2137661/3134462504.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana,inplace=True)


SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [72]:
agrupado = X.groupby(['ESTADOCIVIL']).size()
# Uma possibilidade interessante é usar a Mode, o valor mais comum.
# Podemos agrupar por estado civil e pegar o valor mais comum e usar ele para preencher os nulos.
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [73]:
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

/tmp/ipykernel_2137661/1274388989.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)


SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [74]:
# Uma outra coisa que pode verificar é outliers, valores fora do padrão.
# imagine se você vai criar um modelo de regressão linear utilizando o saldo atual, 
# a sua variável independente é saldo atual.
# Se você tem um alto layer, ou seja, apenas um registro com um valor discrepante, por exemplo,
# um bilionário com o saldo atual dele milhões de vezes acima dos demais, isso vai mudar completamente o seu modelo.
# retirar esse saldo atual para que não afete a previsão de todos osoutros que vão solicitar empréstimo utilizando esse modelo.
desv = X['SALDO_ATUAL'].std()
desv

np.float64(685936688.9820067)

In [75]:
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL'] # Vamos utilizar uma tecnica onde o valor é maior que duas vezes o desvio padrão do saldo atual.

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [76]:
mediana = X['SALDO_ATUAL'].median() # Vamos atualizar as duas linhas acima com a mediana.
mediana

np.float64(2323.0)

In [77]:
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL'] = mediana
X.loc[X['SALDO_ATUAL']>= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


In [78]:
# O que é o Data Binding?
# Remover itens que são duplicados como a digitação errada.
# Então eles significam a mesma coisa, mas foram digitada de forma diferente ou itens que têm pouca relevância.
# então com o database a gente transforma estes essas instâncias com uma única categoria.
# Por exemplo, a gente pode atribuir tudo a outros.
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [79]:
# Pegamos os menos representados e colocamos no grupo Outros.
X.loc[X['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

In [80]:
X['DATA'] # No geral é tratada como categoria, alguns algoritmos podem entender.
# Vamos criar três atributos: dia, mês e ano

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [81]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [82]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [83]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [84]:
X['DIASEMANA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

In [85]:
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [86]:
X['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [87]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [88]:
# No lugard e usar a OneHotEncoding (que criar uma coluna para cada categoria) vamos usar a LabelEncoder
# Isso para não criar muitas colunas pois isso pode aumentar a complexidade do modelo.
# A LabelEncoder transforma cada categoria em um número.
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA']) 

In [89]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [90]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [91]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix = 'OUTROS')
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [92]:
sc = StandardScaler() # Vamos padronizar os dados numericos para que eles fiquem na mesma escala antes de unir os dados de z
m = sc.fit_transform(X.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]], shape=(1000, 3))

In [93]:
X = pd.concat([X,z,pd.DataFrame(m,columns=['SALDO_ATUAL_N','RESIDENCEADESDE_N','IDADE_N'])],axis=1)
# Para m, como não tem nome das colunas e é um objeto do numpy, vamos transformar ele num dataframe e colocar o nome das colunas.

In [94]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,False,True,False,0.572709,1.046987,0.898982


In [95]:
X.drop(columns=['SALDO_ATUAL','RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA','OUTROS_banco'], inplace=True)
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDE_N,IDADE_N
0,3,7,2019,1,5,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,True,False,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,0,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566
997,3,7,2018,7,5,True,False,-0.875522,1.046987,0.105809
998,3,7,2019,7,4,True,False,-0.504738,1.046987,-0.687363
